<a href="https://colab.research.google.com/github/VijayRameshkumar/Deep_Learning/blob/gh-pages/MachineLearning/19_TrickyCNN_TransferLearning/TransferLearning_TrickY_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir /content/dataset

In [ ]:
%cd /content/dataset/

%load_ext tensorboard

/content/dataset


In [ ]:
!mkdir data
!unrar x -r '/content/drive/MyDrive/19_Trasfer Learning/rvl-cdip.rar' ./data

In [ ]:
%cd data/

/content/dataset/data


In [ ]:
%ls

data_final/  labels_final.csv  logs/


In [ ]:
import numpy as np
import pandas as pd
import time
import keras
import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization

from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard, LambdaCallback

import imgaug as ia
from imgaug import augmenters as iaa

[Data Augmentation using IMGAUG and Preprocessing Pipeline](https://www.kaggle.com/mpalermo/keras-pipeline-custom-generator-imgaug)

# 1. Data Analysis

In [ ]:
df = pd.read_csv('/content/dataset/data/labels_final.csv')
# df['label'] = df['label'].astype('str')
df.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from google.colab.patches import cv2_imshow

root = "data_final/"

# width = []
# height = []

# for im_path in df.path:
#   im_path = root + im_path
#   img = cv2.imread(im_path)
#   height.append(img.shape[0])
#   width.append(img.shape[1])

In [ ]:
##plotting 
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(width,bins=20)
plt.xlim(0,1000)
plt.title('width')

In [ ]:
# mean_width = sum(width) // len(width)
# mean_height = sum(height) // len(height)

# print(f"Total {len(width)} of images")
# print(f"min {min(width)}, max {max(width)}, mean_width {mean_width}")
# print(f"min {min(height)}, max {max(height)}, mean_height {mean_height}")

batch_size = 64
step = 48000 / batch_size
print(f"batch_size is {batch_size}")

batch_size is 64


In [ ]:
print(f"labels with counts :\n")

print(df.label.value_counts().sort_index(ascending = True))

labels with counts :

0     3016
1     2994
2     2993
3     3005
4     2994
5     2999
6     2985
7     3000
8     3003
9     3002
10    3002
11    2992
12    3006
13    3007
14    3006
15    2996
Name: label, dtype: int64


#2. Data-Split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(df['path'], df['label'], test_size=0.2, stratify=df['label'])

print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(38400,) (9600,) (38400,) (9600,)


# 2 . Pre-Processing

In [ ]:
import gc

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, images_paths, labels, batch_size=batch_size, 
              image_dimensions = (1000 ,700 ,3), shuffle=False, augment=False):
     
        self.labels       = labels              # array of labels
        self.images_paths = images_paths        # array of image paths
        self.dim          = image_dimensions    # image dimensions
        self.batch_size   = batch_size          # batch size
        self.shuffle      = shuffle             # shuffle bool
        self.augment      = augment             # augment data bool
        self.on_epoch_end()
        gc.collect()
  

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images_paths) / self.batch_size))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        'Generate one batch of data'
        # selects indices of data for next batch
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        root = 'data_final/'

        # select data and load images
        labels = np.array([self.labels[k] for k in indexes])

        images = []
        for k in indexes:
            img = cv2.imread(root + self.images_paths[k])
            img = cv2.resize(img, (1000, 700))
            images.append(img)
        
        # preprocess and augment data
        if self.augment == True:
            images = self.augmentor(images)

        return np.asarray(images), labels
    
    
    def augmentor(self, images):
        'Apply data augmentation'
        sometimes = lambda aug: iaa.Sometimes(0.5, aug)
        seq = iaa.Sequential(
                [
                # apply the following augmenters to most images
                iaa.Fliplr(0.5),  # horizontally flip 50% of all images
                iaa.Flipud(0.2),  # vertically flip 20% of all images
                sometimes(iaa.Affine(
                    # scale images to 80-120% of their size, individually per axis
                    translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                    # translate by -20 to +20 percent (per axis)
                    rotate=(-10, 10),  # rotate by -45 to +45 degrees
                    shear=(-5, 5),  # shear by -16 to +16 degrees
                    order=[0, 1],
                    # use nearest neighbour or bilinear interpolation (fast)
                    cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
                    mode=ia.ALL
                    # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                )),
                # execute 0 to 5 of the following (less important) augmenters per image
                # don't execute all of them, as that would often be way too strong
                iaa.SomeOf((0, 5),
                           [sometimes(iaa.Superpixels(p_replace=(0, 1.0),
                                                             n_segments=(20, 200))),
                               # convert images into their superpixel representation
                               iaa.OneOf([
                                       iaa.GaussianBlur((0, 1.0)),
                                       # blur images with a sigma between 0 and 3.0
                                       iaa.AverageBlur(k=(3, 5)),
                                       # blur image using local means with kernel sizes between 2 and 7
                                       iaa.MedianBlur(k=(3, 5)),
                                       # blur image using local medians with kernel sizes between 2 and 7
                               ]),
                               iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)),
                               # sharpen images
                               iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
                               # emboss images
                               # search either for all edges or for directed edges,
                               # blend the result with the original image using a blobby mask
                               iaa.SimplexNoiseAlpha(iaa.OneOf([
                                       iaa.EdgeDetect(alpha=(0.5, 1.0)),
                                       iaa.DirectedEdgeDetect(alpha=(0.5, 1.0),
                                                              direction=(0.0, 1.0)),
                               ])),
                               iaa.AdditiveGaussianNoise(loc=0,
                                                         scale=(0.0, 0.01 * 255),
                                                         per_channel=0.5),
                               # add gaussian noise to images
                               iaa.OneOf([
                                       iaa.Dropout((0.01, 0.05), per_channel=0.5),
                                       # randomly remove up to 10% of the pixels
                                       iaa.CoarseDropout((0.01, 0.03),
                                                         size_percent=(0.01, 0.02),
                                                         per_channel=0.2),
                               ]),
                               iaa.Invert(0.01, per_channel=True),
                               # invert color channels
                               iaa.Add((-2, 2), per_channel=0.5),
                               # change brightness of images (by -10 to 10 of original value)
                               iaa.AddToHueAndSaturation((-1, 1)),
                               # change hue and saturation
                               # either change the brightness of the whole image (sometimes
                               # per channel) or change the brightness of subareas
                               iaa.OneOf([
                                       iaa.Multiply((0.9, 1.1), per_channel=0.5),
                                       iaa.FrequencyNoiseAlpha(
                                               exponent=(-1, 0),
                                               first=iaa.Multiply((0.9, 1.1),
                                                                  per_channel=True),
                                               second=iaa.ContrastNormalization(
                                                       (0.9, 1.1))
                                       )
                               ]),
                               sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5),
                                                                   sigma=0.25)),
                               # move pixels locally around (with random strengths)
                               sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
                               # sometimes move parts of the image around
                               sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                           ],
                           random_order=True
                           )
                ],
                random_order=True
        )
        return seq.augment_images(images)

NameError: ignored

# Model 1

In [ ]:
class VGG16_TL():
    def __init__(self, image_dimensions=(1000,700 ,3), n_classes=16):
        self.n_classes = n_classes  # number of classes to classify(1 for binary classification)
        self.input_dim = image_dimensions  # image input dimensions
        self.model = self.create_model()  # model

    def summary(self):
        self.model.summary()
  
    def create_model(self,):
        # Loading Model
        pretrained_model = VGG16(input_shape=(1000, 700, 3), include_top=False, weights="imagenet")
        
        # Freezing the layers
        for layer in pretrained_model.layers[:15]:
            layer.trainable = False
        for layer in pretrained_model.layers[15:]:
            layer.trainable = True

        # Modification of pretrained model
        last_layer = pretrained_model.get_layer('block5_pool')
        last_output = last_layer.output

        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block6_conv1')(last_output)
        x = GlobalMaxPooling2D()(x)
        x = Dense(512, activation='relu', name='fc1')(x)
        x = Dropout(0.4)(x)
        x = Dense(216, activation='relu', name='fc2')(x)
        x = Dense(self.n_classes, activation='softmax', name='predictions')(x)

        # Creating a new model
        model = Model(pretrained_model.input, x)

        model.compile(loss='binary_crossentropy',
                optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                metrics=['accuracy'])

        return model

    def train(self, train_data, val_data, plot_results=True):
        'Trains data on generators'
        print("Starting training")

        # reduces learning rate if no improvement are seen
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                                    patience=2,
                                                    verbose=1,
                                                    factor=0.5,
                                                    min_lr=0.0000001)

        # stop training if no improvements are seen
        early_stop = EarlyStopping(monitor="val_loss",
                                   mode="min",
                                   patience=5,
                                   restore_best_weights=True)

        # saves model weights to file
        checkpoint = ModelCheckpoint('./model_weights.hdf5',
                                     monitor='val_loss',
                                     verbose=1,
                                     save_best_only=True,
                                     mode='min',
                                     save_weights_only=True)

        # visualize training data
        tensorboard = TensorBoard(log_dir='./logs',
                                 histogram_freq=0,
                                 batch_size=128,
                                 write_graph=True,
                                 write_grads=True,
                                 write_images=False)

        # reduce resource usage(keeps laptop from melting)
        # idle = LambdaCallback(on_epoch_end=lambda batch, logs: time.sleep(30),
        #                      on_batch_end=lambda batch, logs: time.sleep(0.005))

        # train on data
        history = self.model.fit_generator(generator=train_data,
                                           validation_data=val_data,
                                           epochs=EPOCHS,
                                           steps_per_epoch=600,
                                           validation_steps =150,
                                           callbacks=[learning_rate_reduction, early_stop, checkpoint, tensorboard],
                                           verbose=2,
                                           )
        # plot training history
        if plot_results:
            fig, ax = plt.subplots(2, 1, figsize=(6, 6))
            ax[0].plot(history.history['loss'], label="TrainLoss")
            ax[0].plot(history.history['val_loss'], label="ValLoss")
            ax[0].legend(loc='best', shadow=True)

            ax[1].plot(history.history['acc'], label="TrainAcc")
            ax[1].plot(history.history['val_acc'], label="ValAcc")
            ax[1].legend(loc='best', shadow=True)
            plt.show()

In [ ]:
from keras.utils import np_utils

xtrain = xtrain.reset_index(drop=True).values
xtest = xtest.reset_index(drop=True).values

ytrain = ytrain.reset_index(drop=True).values
ytest = ytest.reset_index(drop=True).values


# One-hot encode data
ytrain = np_utils.to_categorical(ytrain)
ytest = np_utils.to_categorical(ytest)

In [ ]:
def loadData(db, val_split=0.2, sub_sample_size=-1):
    'Loads data into generator object'
    if db == "train":
        train_data = DataGenerator(xtrain, ytrain, batch_size=batch_size, augment=True, shuffle=True)
        val_data = DataGenerator(xtest, ytest, batch_size=batch_size, augment=False, shuffle=False)
        return train_data, val_data

    else:
        return DataGenerator(image_paths, labels, batch_size=BATCH_SIZE, augment=True, shuffle=True), None

if __name__ == "__main__":
    EPOCHS = 10
    BATCH_SIZE = 64
    IMAGE_DIMENSIONS = (1000,700 ,3)

    # create model
    model = VGG16_TL(image_dimensions=IMAGE_DIMENSIONS, n_classes=16)
    model.summary()

    # train model
    train_data, val_data = loadData("train", val_split=0.2)
    model.train(train_data, val_data, plot_results=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 700, 3)]    0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 1000, 700, 64)     1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 1000, 700, 64)     36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 500, 350, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 500, 350, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 500, 350, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 250, 175, 128)     0     

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
